In [96]:
"|IMPORT PACKAGES|"
import numpy              as np
import pandas             as pd
import datetime
from   math               import pi
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Text, Span
from   bokeh.models.tools import HoverTool
from   bokeh.models       import Arrow, NormalHead, OpenHead, VeeHead
from   bokeh.transform    import dodge
from   datetime           import datetime as dt

In [97]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'

IO   = path + file

sheet = 'MONTHLY'

data = pd.read_excel(IO, sheet_name = sheet, usecols="B,KH,KL,KN", skiprows=2, nrows=396, engine='openpyxl') # Be patient...

data = data.rename(columns={"INDEC [POBREZA POBLACION]"   :"INDEC_1"})
data = data.rename(columns={"CEDLAS [POBREZA POBLACION]":"CEDLAS_1"})
data = data.rename(columns={"UCA [POBREZA HOGARES]"     :"UCA_1"})

data['INDEC_2']  = data['INDEC_1'].interpolate()
data['CEDLAS_2'] = data['CEDLAS_1'].interpolate()
data['UCA_2']    = data['UCA_1'].interpolate()

data['INDEC_2'][228:341] = np.NaN # Eliminar datos interpolados durante intervencion K del INDEC
data['UCA_2'][384:] = np.NaN      # Eilimnar interpolación innecesaria al final de la serie
data['CEDLAS_2'][372:] = np.NaN   # Eilimnar interpolación innecesaria al final de la serie

<ipython-input-97-6216bd5eec56>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['INDEC_2'][228:341] = np.NaN # Eliminar datos interpolados durante intervencion K del INDEC
<ipython-input-97-6216bd5eec56>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['UCA_2'][384:] = np.NaN      # Eilimnar interpolación innecesaria al final de la serie
<ipython-input-97-6216bd5eec56>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data

In [71]:
"|CHECK DATA|"
data.tail(50)

,MONTH,INDEC_1,CEDLAS_1,UCA_1,INDEC_2,CEDLAS_2,UCA_2
346,2016-11-30,NaN,NaN,NaN,29.816667,22.366667,32.566667
347,2016-12-31,29.6,NaN,32.8,29.600000,22.300000,32.800000
348,2017-01-31,NaN,NaN,NaN,29.466667,22.233333,32.416667
349,2017-02-28,NaN,NaN,NaN,29.333333,22.166667,32.033333
350,2017-03-31,NaN,NaN,NaN,29.200000,22.100000,31.650000
351,2017-04-30,NaN,NaN,NaN,29.066667,22.033333,31.266667
352,2017-05-31,NaN,NaN,NaN,28.933333,21.966667,30.883333
353,2017-06-30,28.8,21.9,NaN,28.800000,21.900000,30.500000
354,2017-07-31,NaN,NaN,NaN,28.250000,21.866667,30.116667
355,2017-08-31,NaN,NaN,NaN,27.700000,21.833333,29.733333


In [98]:
"|BUILD PLOT|"

cds = ColumnDataSource(data)

data['MONTH'] = pd.to_datetime(data['MONTH'])

#BUILD FIGURE
p = figure(title       = "EL HUB ECONÓMICO | PERSONAS BAJO LA LÍNEA DE POBREZA",
           y_range     = (0, 60),
           x_axis_type = "datetime", 
           plot_height = 400,
           plot_width  = 700)

p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
p.toolbar_location = "right"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
p1 = p.line("MONTH", "INDEC_2" , color='black', line_alpha=0.5, width=2, legend_label="INDEC" , muted_alpha=0.20, line_dash='solid', source=cds)
p2 = p.line("MONTH", "CEDLAS_2", color='red'  , line_alpha=0.5, width=2, legend_label="CEDLAS", muted_alpha=0.20, line_dash='solid', source=cds)
p3 = p.line("MONTH", "UCA_2"   , color='blue' , line_alpha=0.5, width=2, legend_label="UCA"   , muted_alpha=0.20, line_dash='solid', source=cds)

p4 = p.circle('MONTH', 'INDEC_1' , color='black', fill_alpha=0.5, line_alpha=0.5, size=7, source=cds)
p5 = p.circle('MONTH', 'CEDLAS_1', color='red'  , fill_alpha=0.5, line_alpha=0.5, size=7, source=cds)
p6 = p.circle('MONTH', 'UCA_1'   , color='blue' , fill_alpha=0.5, line_alpha=0.5, size=7, source=cds)

p.add_tools(HoverTool(renderers=[p4], tooltips = [("INDEC" , "@INDEC_1{0.0}")] , mode="vline"))
p.add_tools(HoverTool(renderers=[p5], tooltips = [("CEDLAS", "@CEDLAS_1{0.0}")], mode="vline"))
p.add_tools(HoverTool(renderers=[p6], tooltips = [("UCA"   , "@UCA_1{0.0}")]   , mode="vline"))

#INTERVENCION AL INDEC
Intervencion = BoxAnnotation(left=dt(2007, 1,31).timestamp()*1000, right=dt(2016, 5,31).timestamp()*1000, bottom=0, line_color="black" ,fill_color="gray", fill_alpha=0.1)

x = (dt(2007, 1,31).timestamp()*1000 + dt(2016, 5,31).timestamp()*1000)/2
Intervencion_N = Label(x=x, y=55, text="Intervención al INDEC", text_font_size="10pt", text_align="center", text_baseline='bottom', angle_units="deg", angle= 0)

p.add_layout(Intervencion)
p.add_layout(Intervencion_N)

#MARCAS
hiperinflacion_text = Label(x=dt(1991, 1, 31).timestamp()*1000, y=2, text_font_size='12px', text_color='green', text_align='center', text_baseline='bottom',
                    text='Fin hiperinflación')
hiperinflacion_arrow = Arrow(end=VeeHead(line_color='green', line_width=2, line_alpha=0.5, size=10, fill_color='green', fill_alpha=0.75), line_color='green',
                     x_start=dt(1991, 1,31).timestamp()*1000, x_end=dt(1991, 1,31).timestamp()*1000, y_start=5, y_end=27)

tequila_text = Label(x=dt(1995, 1,31).timestamp()*1000, y=35, text_font_size='12px', text_color='green', text_align='center', text_baseline='bottom',
                    text='Crisis Tequila')
tequila_arrow = Arrow(end=VeeHead(line_color='green', line_width=2, line_alpha=0.5, size=10, fill_color='green', fill_alpha=0.75), line_color='green',
                     x_start=dt(1995, 1,31).timestamp()*1000, x_end=dt(1995, 1,31).timestamp()*1000, y_start=33, y_end=24)

recesion98_text = Label(x=dt(1998, 5,31).timestamp()*1000, y=42, text_font_size='12px', text_color='green', text_align='center', text_baseline='bottom',
                    text='Inicio recesión 1998')
recesion98_arrow = Arrow(end=VeeHead(line_color='green', line_width=2, line_alpha=0.5, size=10, fill_color='green', fill_alpha=0.75), line_color='green',
                     x_start=dt(1998, 5,31).timestamp()*1000, x_end=dt(1998, 5,31).timestamp()*1000, y_start=40, y_end=28)

crisis2001_text = Label(x=dt(2001,12, 31).timestamp()*1000, y=2, text_font_size='12px', text_color='green', text_align='center', text_baseline='bottom',
                    text='Crisis 2001')
crisis2001_arrow = Arrow(end=VeeHead(line_color='green', line_width=2, line_alpha=0.5, size=10, fill_color='green', fill_alpha=0.75), line_color='green',
                     x_start=dt(2001,12,31).timestamp()*1000, x_end=dt(2001,12,31).timestamp()*1000, y_start=5, y_end=30)

estanflacion_text = Label(x=dt(2012, 1, 31).timestamp()*1000, y=2, text_font_size='12px', text_color='green', text_align='center', text_baseline='bottom',
                    text='Inicio estanflación')
estanflacion_arrow = Arrow(end=VeeHead(line_color='green', line_width=2, line_alpha=0.5, size=10, fill_color='green', fill_alpha=0.75), line_color='green',
                     x_start=dt(2012, 1,31).timestamp()*1000, x_end=dt(2012, 1,31).timestamp()*1000, y_start=5, y_end=22)

crisis_fx_text = Label(x=dt(2018, 4, 1).timestamp()*1000, y=2, text_font_size='12px', text_color='green', text_align='center', text_baseline='bottom',
                    text='Crisis cambiaria')
crisis_fx_arrow = Arrow(end=VeeHead(line_color='green', line_width=2, line_alpha=0.5, size=10, fill_color='green', fill_alpha=0.75), line_color='green',
                     x_start=dt(2018, 4, 1).timestamp()*1000, x_end=dt(2018, 4, 1).timestamp()*1000, y_start=5, y_end=20)

p.add_layout(hiperinflacion_text)
p.add_layout(hiperinflacion_arrow)
p.add_layout(tequila_text)
p.add_layout(tequila_arrow)
p.add_layout(recesion98_text)
p.add_layout(recesion98_arrow)
p.add_layout(crisis2001_text)
p.add_layout(crisis2001_arrow)
p.add_layout(estanflacion_text)
p.add_layout(estanflacion_arrow)
p.add_layout(crisis_fx_text)
p.add_layout(crisis_fx_arrow)


#LEGEND
p.legend.location     = "top_left"
p.legend.orientation  = "horizontal"
p.legend.click_policy = "mute"

show(p)

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="pobreza.png")

In [92]:
"|CREATE HTML FILE|"

output_file(filename="pobreza.html", title="Personas bajo la línea de pobreza")
save(p)

'C:\\Users\\ncach\\OneDrive\\My Documents\\Research\\GitHub Repositories\\el-hub-economico\\static\\Jupyter Notebooks\\pobreza.html'

In [95]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "pobreza"))
with open("pobreza.json", "w") as fp:
    json.dump(j, fp)